In [1]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [2]:
def open_file(file):
    with open(file) as f:
        program = [line[: -1] for line in f]
    return program

In [3]:
def add_hex(num1, num2):
    return hex(int(num1, 16) + int(num2, 16))[2: ].upper()

In [4]:
def sub_hex(num1, num2):
    answer = hex(int(num1, 16) - int(num2, 16)).upper()
    if answer[0] == '-':
        return answer[3: ]
    return answer[2: ]

In [5]:
def sub_hex(num1, num2):
    answer = hex(int(num1, 16) - int(num2, 16)).upper()
    if answer[0] == '-':
        return '-' + answer[3: ]
    return answer[2:]

In [6]:
def get_prog_dimension(head_record):
    start = head_record[9: 13]
    length = head_record[15: ]
    return start, add_hex(start, length)

In [7]:
def get_memory_dimension(start, length):
    memory_start = start[: 3] + str(0)
    memory_end = length[: 2] + hex(int(length[3], 16) + 1)[2: ].upper() + str(0)
    return memory_start, memory_end

In [51]:
def get_memory_addresses(memory_start, memory_end):
    memory_addresses = []
    while(int(memory_start, 16) <= int(memory_end, 16)):
        memory_addresses.append(memory_start.zfill(4))
        memory_start = add_hex(memory_start, '10')
    return memory_addresses

In [52]:
def get_text_records(program):
    text_records = []
    for line in program:
        if line[0] == 'T':
            text_records.append(text_record(line))
    return text_records

In [53]:
def text_record(text_record):
    start = text_record[3: 7]
    values = re.findall('..?', text_record[9: ])
    return {
        'Start': start,
        'Values': values
    }

In [54]:
program = open_file('sic.txt')
start, end = get_prog_dimension(program[0])

In [55]:
memory_start, memory_end = get_memory_dimension(start, end)

In [56]:
memory_addresses = get_memory_addresses(memory_start, memory_end)

In [57]:
memory_graph = pd.DataFrame(columns = ['Memory_Address', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F'])

In [58]:
memory_graph.Memory_Address = memory_addresses

In [59]:
memory_graph.fillna('xx', inplace = True)

In [60]:
text_records = get_text_records(program)


In [99]:
def memory_allocation(text_records, memory_graph, starting_address):
    for record in text_records:
        record['Start'] = add_hex(record['Start'], starting_address).zfill(4)
        
        start = record['Start'][:-1] + '0'
        column = record['Start'][-1]
        values = record['Values']
        index = memory_graph[memory_graph.Memory_Address == start].index

        for value in values:
            memory_graph.loc[index, column] = value
            column = add_hex(column, '1')

            if column == '10':
                index += 1
                column = '0'
    return memory_graph

In [100]:
memory_graph = memory_allocation(text_records, memory_graph, '0000')
memory_graph

,Memory_Address,0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F
0,0000,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
1,0010,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
2,0020,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
3,0030,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
4,0040,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
5,0050,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
6,0060,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
7,0070,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
8,0080,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
9,0090,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx


In [101]:
program = open_file('sicxe.txt')


In [102]:
def control_section_head_info(head_record):
    name = head_record[1: 7].strip()
    length = head_record[15: ]
    return name, length

In [103]:
def control_section_defintion_reference(definition, reference):
    definition = definition[1: ]
    definition = re.findall('......?', definition)
    i = 0
    definitions = []
    while i < len(definition):
        symbol = definition[i].strip()
        address = definition[i+1][2:]
        definitions.append({
            'Symbol': symbol,
            'Address': address
        })
        i += 2
    
    reference = reference[1: ]
    reference = re.findall('......?', reference)
    references = [var.strip() for var in reference]
    
    return definitions, references

In [104]:
def control_section_modification(control_section):
    modification_list = []
    for line in control_section:
        if line[0] == 'M':
            address = line[3: 7]
            size = line[7: 9]
            operation = line[9: ]
            modification_list.append({
                'Address': address,
                'Size': size,
                'Operation': operation
            })
            
    return  modification_list

In [105]:
def control_sections(program):
    control_sections = []
    control_section = []
    for line in program:
        control_section.append(line)
        if(line[0] == 'E'):
            name, length = control_section_head_info(control_section[0])
            definitions, references = control_section_defintion_reference(control_section[1], control_section[2])
            t_records = get_text_records(control_section)
            modification_list = control_section_modification(control_section)
            control_section_obj = {
                'Name': name,
                'Length': length,
                'Definitions': definitions,
                'References': references,
                'T-Records' : t_records,
                'M-Records': modification_list
            }
            control_sections.append(control_section_obj)
            control_section = []

    return control_sections

In [106]:
control_sections = control_sections(program)

In [107]:
starting_address = '0000'

In [108]:
def get_estab_df(control_sections, starting_address):
    dict = []
    total_length = starting_address
    for control_section in control_sections:
        name = control_section['Name']
        address = starting_address
        dict.append({
            'Control_Section':name,
            'Symbol_Name': ' ',
            'Address': address.zfill(4),
            'Length': control_section['Length']
        })
        total_length = add_hex(total_length, control_section['Length'])
        
        for definition in control_section['Definitions']:
            symbol = definition['Symbol']
            address = add_hex(definition['Address'], starting_address)
            
            dict.append({
                'Control_Section':' ',
                'Symbol_Name': symbol,
                'Address': address.zfill(4),
                'Length': ' '
                })
        starting_address = add_hex(starting_address, control_section['Length'])
    return pd.DataFrame(dict), total_length.zfill(4)
    

In [109]:
def get_estab_dict(estab_df):
    list = {}
    for i in range(len(estab_df)):
        line = estab_df.iloc[i]
        if(line.Control_Section == ' '):
            symbol = line.Symbol_Name
        else:
            symbol = line.Control_Section
        address = line.Address
        list[symbol] = address
        
    return list

In [110]:
estab_df, end = get_estab_df(control_sections, starting_address)
estab_df

,Control_Section,Symbol_Name,Address,Length
0,PROGA,,0000,0063
1,,LISTA,0040,
2,,ENDA,0054,
3,PROGB,,0063,007F
4,,LISTB,00C3,
5,,ENDB,00D3,
6,PROGC,,00E2,0051
7,,LISTC,0112,
8,,ENDC,0124,


In [111]:
estab_dict = get_estab_dict(estab_df)
estab_dict

{'PROGA': '0000',
 'LISTA': '0040',
 'ENDA': '0054',
 'PROGB': '0063',
 'LISTB': '00C3',
 'ENDB': '00D3',
 'PROGC': '00E2',
 'LISTC': '0112',
 'ENDC': '0124'}

In [112]:
memory_graph = pd.DataFrame(columns = ['Memory_Address', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F'])

In [113]:
memory_start, memory_end = get_memory_dimension(starting_address, end)

In [114]:
memory_addresses = get_memory_addresses(memory_start, memory_end)

In [115]:
memory_graph.Memory_Address = memory_addresses

In [116]:
memory_graph.fillna('xx', inplace = True)

In [117]:
for control_section in control_sections:
    memory_graph = memory_allocation(control_section['T-Records'], memory_graph, estab_dict[control_section['Name']])
memory_graph

,Memory_Address,0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F
0,0000,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
1,0010,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
2,0020,03,20,1D,77,10,00,04,05,00,14,xx,xx,xx,xx,xx,xx
3,0030,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
4,0040,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
5,0050,xx,xx,xx,xx,00,00,14,FF,FF,F6,00,00,3F,00,00,14
6,0060,FF,FF,C0,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
7,0070,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
8,0080,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
9,0090,xx,xx,xx,xx,xx,xx,xx,xx,xx,03,10,00,00,77,20,27


In [118]:
def memory_modifier(memory_graph, address, size, value):
    start = address[:-1]+ '0'
    column = address[-1]
    old_column = column
    index = memory_graph[memory_graph.Memory_Address == start].index[0]
    old_index = index
    old_value = ' '
    for i in range(3):
        old_value += memory_graph.loc[index, column]
        column = add_hex(column, '1')
        if column == '10':
            index += 1
            column = '0'
            
    old_value = old_value.strip()
        
    column = old_column
    index = old_index
    
    symbol = operation[1: ]
    operand = estab_dict[symbol]
    if(size == '05'):
        if(operation[0] == '+'):
            new_value = old_value[0] + add_hex(old_value[1: ], operand).zfill(5)
        else:
            new_value = old_value[0] + sub_hex(old_value[1: ], operand).zfill(5)
            
    else:
        if(operation[0] == '+'):
            new_value = add_hex(old_value, operand)
        else:
            new_value = sub_hex(old_value, operand)
            
    
    new_value = new_value.zfill(6)
    #Overflow
    if len(new_value) == 7:
        new_value = new_value[1: ]
        
    new_value = re.findall('..?', new_value)

    for i in range(3):
        memory_graph.loc[index, column] = new_value[i]
        
        column = add_hex(column, '1')
        if column == '10':
            index += 1
            column = '0'
    
    return memory_graph


In [121]:
for control_section in control_sections:
    name = control_section['Name']
    starting_address = estab_dict[name]
    m_records = control_section['M-Records']
    for record in m_records:
        address = add_hex(record['Address'], starting_address).zfill(4)
        size = record['Size']
        operation = record['Operation']
        memory_graph = memory_modifier(memory_graph, address, size, operation)
    

In [123]:
memory_graph

,Memory_Address,0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F
0,0000,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
1,0010,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
2,0020,03,20,1D,77,10,00,C7,05,00,14,xx,xx,xx,xx,xx,xx
3,0030,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
4,0040,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
5,0050,xx,xx,xx,xx,00,01,26,00,00,08,00,00,51,00,00,04
6,0060,00,00,83,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
7,0070,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
8,0080,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx
9,0090,xx,xx,xx,xx,xx,xx,xx,xx,xx,03,10,00,40,77,20,27


In [124]:
estab_dict

{'PROGA': '0000',
 'LISTA': '0040',
 'ENDA': '0054',
 'PROGB': '0063',
 'LISTB': '00C3',
 'ENDB': '00D3',
 'PROGC': '00E2',
 'LISTC': '0112',
 'ENDC': '0124'}